## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
#載入套件
from keras.models import Sequential  #用來啟動 NN
from keras.layers import Convolution2D # Convolution Operation
from keras.layers import MaxPooling2D # Pooling
from keras.layers import Flatten
from keras.layers import Dense # Fully Connected Networks
from keras.layers import Dropout 
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
# CUDNN_STATUS_INTERNAL_ERROR 於運行時出現, 重裝 cudnn 之後無效, suspect gpu ram issue
# 利用動態分配GPU內存 problem solved.
# ref: https://github.com/keras-team/keras/issues/4161
def dynam_GPU_ram():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
    config.log_device_placement = True  # to log device placement (on which device the operation ran)
    sess = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

    
    
dynam_GPU_ram()

config = tf.ConfigProto(allow_soft_placement=True) 
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
'''
您好，我們常見的前處理包含Normalization 和 Standardization，
狹義的Normalization 指的是將所有資料rescale到0-1之間，
在影像中/255就是其中一種，上方程式部分比較正確的名稱是Standardization，
也就是將所有的資料-平均值/STD，之所以會加上一個很小的值是為了避免分母為0，
算是廣義的Normalization ，這種方式結果有正有負。
'''
def normalize(X_train,X_test):
        #計算平均值
        mean = np.mean(X_train,axis=(0,1,2,3)) #np.mean(axis=(0,1,2,3))輸出的矩陣維度為4維。
        #計算標準差
        std = np.std(X_train, axis=(0, 1, 2, 3)) #np.std(axis=(0,1,2,3))輸出的矩陣維度為4維。
        #透過平均值與標準差將X_train的資料歸一化
        X_train = (X_train-mean)/(std+1e-7)  #加上一個很小的值(1e-7)，為了避免分母為0。
        #透過平均值與標準差將X_test的資料歸一化
        X_test = (X_test-mean)/(std+1e-7)  #加上一個很小的值(1e-7)，為了避免分母為0。
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
# 對 label 進行 one-hot encoding 
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
# 建立簡單的線性執行的模型
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters= 32,kernel_size=(3,3),input_shape=(32,32,3),activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2))) # 建立池化層，池化大小=2x2，取最大值

#卷積組合
classifier.add(Convolution2D(filters= 32,kernel_size=(3,3),activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax')) # 使用 softmax activation function，並將結果分類

#超過兩個就要選categorical_crossentrophy
# 編譯: 選擇損失函數、優化方法及成效衡量方式
# 確立目標及求解方法：以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 8s 162us/step - loss: 1.3466 - accuracy: 0.5470
Epoch 2/100
50000/50000 [==============================] - 6s 127us/step - loss: 0.8164 - accuracy: 0.7144
Epoch 3/100
50000/50000 [==============================] - 6s 128us/step - loss: 0.5941 - accuracy: 0.7921
Epoch 4/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.4198 - accuracy: 0.8546
Epoch 5/100
50000/50000 [==============================] - 6s 124us/step - loss: 0.2765 - accuracy: 0.9065
Epoch 6/100
50000/50000 [==============================] - 6s 124us/step - loss: 0.1944 - accuracy: 0.9343
Epoch 7/100
50000/50000 [==============================] - 6s 127us/step - loss: 0.1511 - accuracy: 0.9480
Epoch 8/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.1343 - accuracy: 0.9514
Epoch 9/100
50000/50000 [==============================] - 6s 128us/step - loss: 0.1065 - accuracy: 0.9633
Epoch 10/100
50000/50000 [===========

Epoch 77/100
50000/50000 [==============================] - 7s 136us/step - loss: 0.0143 - accuracy: 0.9958
Epoch 78/100
50000/50000 [==============================] - 7s 133us/step - loss: 0.0145 - accuracy: 0.9953
Epoch 79/100
50000/50000 [==============================] - 7s 134us/step - loss: 0.0132 - accuracy: 0.9955
Epoch 80/100
50000/50000 [==============================] - 7s 130us/step - loss: 0.0120 - accuracy: 0.9963
Epoch 81/100
50000/50000 [==============================] - 6s 128us/step - loss: 0.0197 - accuracy: 0.9942
Epoch 82/100
50000/50000 [==============================] - 6s 128us/step - loss: 0.0174 - accuracy: 0.9948
Epoch 83/100
50000/50000 [==============================] - 6s 128us/step - loss: 0.0101 - accuracy: 0.9967
Epoch 84/100
50000/50000 [==============================] - 6s 129us/step - loss: 0.0153 - accuracy: 0.9950
Epoch 85/100
50000/50000 [==============================] - 6s 127us/step - loss: 0.0147 - accuracy: 0.9952
Epoch 86/100
50000/50000 [==

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [6]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7)  #np.zeros 可以用來建立以 0 為元素的矩陣
classifier.predict(input_example)

array([[1.0075809e-16, 7.4300616e-24, 8.5674579e-09, 9.9999976e-01,
        2.8860200e-07, 3.2877883e-24, 1.2188357e-20, 8.5895301e-28,
        4.6444429e-14, 9.5636281e-33]], dtype=float32)